# Computing the Euclidean Distance Matrix with NumPy

In this notebook we implement a function to compute the Euclidean distance matrix. We use a simple algebra trick that makes possible to write the function in a completely vectorized way in terms of optimized NumPy functions. We  compare this function to a naive implementation that uses loops to combine all rows into the distance matrix.

In [1]:
import numpy as np

In [2]:
def euclidean_loops(x, y):
    """Euclidean square distance matrix
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """
    num_samples = x.shape[0]
    dist_matrix = np.zeros((num_samples, num_samples))
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            diff = xi - yj
            dist_matrix[i][j] = np.dot(diff, diff)
    return dist_matrix

In [3]:
def euclidean_numpy(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N,) numpy array
    y: (N,) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """

    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

## The `euclidean_numpy` function

Each element of the Euclidean distance matrix is the scalar product of the diference between two rows fo the dataset. `euclidean_numpy` takes advantage of this by doing the following
$$\sum_k {(x_{ik}-y_{ik})^2} = (\vec{x}_i - \vec{y}_j)\cdot(\vec{x}_i - \vec{y}_j) = \vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j - 2\vec{x}_i\cdot\vec{y}_j$$

There are NumPy functions to compute each of these terms:

$\vec{x}_i\cdot\vec{y}_j$ $\rightarrow$ `np.dot(x, y)` : Matrix product of $\{\vec{x}\}$ and $\{\vec{y}\}$

$\vec{x}_i\cdot\vec{x}_i$ $\rightarrow$ `np.einsum('ij,ij->i', x, x)[:,np.newaxis]` : A $(n,1)$ vector of elements $\sum_j x_{ij}x_{ij}$

$\vec{y}_j\cdot\vec{y}_j$ $\rightarrow$ `np.einsum('ij,ij->i', y, y)[:,np.newaxis].T` : A $(1,n)$ vector of elements $\sum_j y_{ij}y_{ij}$

To have all the combinations $ij$ of the sum $\vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j$, we add a new axis to each of the arrays, transpose one them and add them.

Let's see now how the `np.einsum` function works. `einsum` stands for Einstein summation, which is used in tensor algebra to write compact expressions without the sum symbol ($\sum$). Within the Einstein summation notation, whenever there are repeated indexes, there is a sum over them. For instance, the expression
$$x_{ik}y_{kj}$$
is equivalent to
$$\sum_k x_{ik}y_{kj}$$

`np.einsum` uses a generalized form of the Einstein summation by adding the symbol `->` to prevent summing over certain indexes. The specific operation we use here, `np.einsum('ij,ij->i', x, x)`, gives the vector
$$
\begin{bmatrix}
\sum_k x_{1k}x_{1k} \\
\sum_k x_{2k}x_{2k} \\
 ...                \\
\sum_k x_{nk}x_{nk} \\
\end{bmatrix}
$$
Note that the resulting vector is represented here as a column vector just for visualization purposes. It's is `(n,)` NumPy array.

Let's check now step-by-step what the `euclidean_numpy` function does:

In [4]:
# Lets generate some random data
nsamples = 10
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

In [5]:
x2 = np.einsum('ij,ij->i', x, x)
x2.shape

(10,)

In [6]:
x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
x2.shape

(10, 1)

In [7]:
(x2 + x2.T).shape

(10, 10)

We now use `np.dot` to perform the matrix multiplication of the full dataset by itself. We didn't use it before as alternative to `np.einsum` because it doesn't perform row by row scalar products. Instead `np.dot` expects two arrays with matching shapes $(m,n)$ and $(n,m)$ to perform a matrix multiplication.

We could have used `np.einsum('ik,jk', x, x)` to perform the matrix multiplication, but we chose `np.dot(x, x.T)` instead. This is because `np.dot` is a very sophisticated too, plus it uses OpenMP threads. This results in a very fast execution.

You are wellcome to time them and look at the `top` command to see how `np.dot` uses multiple OpenMP threads.

In [8]:
xy = np.dot(x, x.T)
xy.shape

(10, 10)

Now, considering that the reason we are using `np.einsum` is to get rid of the loops, why didn't we use something like `(x*x).sum(axis=1)`? Let's run the next cell comparing them:

In [9]:
# let's use a larger array for timing the function calls
nsamples = 1000
nfeat = 300

x = 10. * np.random.random([nsamples, nfeat])

# it gives the same result
np.abs(np.einsum('ij,ij->i', x, x) - (x*x).sum(axis=1)).max()

# but it's not as fast as `np.einsum`
%timeit np.einsum('ij,ij->i', x, x)
%timeit (x*x).sum(axis=1)

128 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
244 µs ± 59.1 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Doing a reduction with the ufunc `np.add` is also slower than `np.einsum`

In [10]:
%timeit np.add.reduce(x*x, axis=1)

# As homework, check what's the meaning of the previous line!

242 µs ± 97.9 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Finally, let's time both implementations!

In [11]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_loops(x, x)
%timeit euclidean_numpy(x, x)

5.76 s ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
48.2 ms ± 25.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Conclusions

The main points to take from this notebook are:
  * Loops in python must be avoided.
  * NumPy is all about vectorization.
  * Always consider different vectorized implementations and compare them. Even within NumPy some functions might bring a more significant speedup than others.